In [44]:
import openai
from dotenv import dotenv_values
import logging
import uuid
from datetime import datetime

# Reading the .env file and storing the values in a variables
config = dotenv_values(".env")

# # Setting up the Open AI properties.
#GPT 3
# openai.api_key = "4e21add22e3e4718a72cadd8d78a0336"
# openai.api_type = "azure"
# openai.api_base = "https://dbs-tmo-dev.openai.azure.com/"
# openai.api_version = "2024-02-15-preview"
# deployment_name = "DBS-TMO-DEV"
# model = "gpt-35-turbo-16k"

#GPT 4
openai.api_key = "3f5dd45a468d4304aeebb8db0dcd672e"
openai.api_type = "azure"
openai.api_base ="https://dbs-tmo-poc-gpt4.openai.azure.com/"
openai.api_version =  "2024-02-15-preview"
deployment_name = "DBS-TMO-POC-GPT4-32K"
model =  "gpt-4-32k"

# Model tuning parameters
temperature=1
top_p=1




In [45]:
# Configure logging
logging.basicConfig(filename='user_story.log', level=logging.INFO, format='%(message)s')

def log_message(user_id, message):
    log_entry = {
        "id": str(uuid.uuid4()),
        "user_id": user_id,
        "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "message": message
    }
    logging.info(log_entry)
    # Convert to timestamp

In [46]:
org_requirements = str("""
Summary:
Decommission Old Data Platform - RECON job
Description :
Today for RECON job, we are using OLD SYSTEM to extract the data from AUX_BILLING_ACCOUNT, format data into specific layout and transfer to NEW ANALYTICS. This job is scheduled at 5 PM IST daily.
 Since OLD SYSTEM is going away and the plan is to move the entire process to AZURE cloud and leverage data bricks to extract the data from new data source.
 *Acceptance Criteria:*
 As a PO I would like to extract the RECON data from new data source. Format data into Vendor suggested layout. Encrypt recon file. Transfer to NEW ANALYTICS
  So that NEW can perform write off account Reconciliation.
 *Scope:*
 Pull the data from database tables.  
Extracting the data into Vendor suggested format and creating a text file with format "Recon_DDMMYYYY.txt". 
 Encrypt reconciliation file (Format - " Recon_DDMMYYYY.txt")
 Transfer the file to vendor. ([abc.sftp.newanalytics.net|http://abc.sftp.newanalytics.net] /Inbox)
 Perform A=B Validation
 Set up Alerts and Notifications - Notify Alerts and notifications on both Success and Failure.
ADF+ADB pipeline 
1.	Team is building an Azure based Solution (ADF+ADB) where it extracts data from new data source, formats data into Vendor specific layout and encrypt recon file using the New Analytics Public Key, places the files in the below specific storage.
2. Control-M will pick the encrypted file from Azure specific Storage and transfer files to New Analytics
3. Control-M should not delete the file once the file is sent to the Vendor during the Soft Launch Mode and should be deleted after business confirms the change is good
""")



In [47]:
# analysis and gap identification for the input feature details.
temperature=0.65
top_p=1

gap_system_prompt = str("""
Please act as a technical architect working for T-Mobile which is a large Telecom provider based out of North America 
and you are provided with high-level designs for a system change that needs to happen. 
This high-level design is provided by the customer architects in the form of a Jira feature. 
Your task is to do an analysis of the design details provided and identify any gaps or missing details. 
Please consider the following aspects which identify the gaps.
Any terms defined but not elaborated.
Any terms which are not understood by you.
Any gaps in the requirements 
Any detail provided that is not clear.
Any other suggestion.                                  
<jira-feature-details>
 """ + org_requirements + """                   
</jira-feature-details> 
                                              
""")

gap_user_prompt = str("""
 Pelase identify the gaps in the given requirements and provide the suggestions to fill the gaps.
""")

messages = [
    {"role": "system", "content": gap_system_prompt},    
    {"role": "user", "content": gap_user_prompt}
]

code = [""]

#request_id = generate_unique_id()
log_message('user_ag', messages)


result=openai.ChatCompletion.create(
#result=openai.Completion.create(
   model= model, 
   messages=messages,
   engine=deployment_name,
    temperature=temperature,
   top_p=top_p
   
)

print("---- Tokens------")
# print(f"completion tokens- {result.usage.completion_tokens}")
# print(f"prompt tokens- {result.usage.prompt_tokens}")
# print(f"Total tokens- {result.usage.total_tokens}")

res = result.choices[0].message.content
gap_assistant = str(res)
# print(f"Response: {res}")
code.append(f"{res}")

---- Tokens------


In [48]:
# Rewriting the input requirements using Gen AI
temperature=0.65
top_p=1

rewrite_input_system_prompt = str("""
Please act as a technical product owner working for T-Mobile which is a large Telecom provider based out of North America 
and you are provided with high-level designs for a system change that needs to happen. 
This high-level design is provided by the customer's architects in the form of a Jira feature, in which various details are seprated by ---. 

Your task is to format and rewrite the inputs provided using the below guidelines
1. Please augment the writing with your expertise in the subject while rewriting the content.
2. Please use a US English dictionary.
3. Make use of moderately complex words.
4. Keep language simple and easy to read.
5. Please use a professional tone while rewriting.
6. Please add #ToDo where missing details can be added by the customer's technical architect.  
                             
<jira-feature-details>
 """ + org_requirements + """                   
</jira-feature-details> 
                                              
""")

rewrite_input_user_prompt = str("""
 Pelase help rewrite the inputs.
""")

messages = [
    {"role": "system", "content": rewrite_input_system_prompt},    
    {"role": "user", "content": rewrite_input_user_prompt}
]

code = [""]

result=openai.ChatCompletion.create(
# result=openai.Completion.create(
   model= model, 
   messages=messages,
   engine=deployment_name,
    temperature=temperature,
   top_p=top_p
   
)

print("---- Tokens------")
print(f"completion tokens- {result.usage.completion_tokens}")
print(f"prompt tokens- {result.usage.prompt_tokens}")
print(f"Total tokens- {result.usage.total_tokens}")

res = result.choices[0].message.content
rewrite_updated_input_assistant = str(res)
print(f"Response: {res}")
code.append(f"{res}")

---- Tokens------
completion tokens- 409
prompt tokens- 552
Total tokens- 961
Response: <jira-feature-details>

Summary: 
Phase Out of Legacy Data Platform - RECON Job Migration

Description:
Our current RECON job utilizes the soon-to-be-decommissioned OLD SYSTEM to extract data from the AUX_BILLING_ACCOUNT. This data is then formatted into a specific layout and transferred to NEW ANALYTICS, a process that occurs daily at 5 PM IST. The imminent retirement of the OLD SYSTEM necessitates a system migration to the AZURE cloud. We plan to harness the power of data bricks for extracting data from a new data source.

Acceptance Criteria:
As a Product Owner, I envision the extraction of RECON data from the new data source, with the data formatted according to the vendor's suggested layout. This data should then be encrypted into a recon file and transferred to NEW ANALYTICS, facilitating a smooth transition for account reconciliation.

Scope:
The process entails extracting data from database 

In [49]:
# Tasks / Steps from summerization to implement the technical change.
temperature=0.65
top_p=1

tasks_system_prompt = str("""
Please act as a technical architect working for T-Mobile which is a large Telecom provider based out of North America
and you are provided with a high-level summary of a system change that needs to happen. 
Your task is to create technical tasks that need to be performed to complete the system change. 

While creating the task list please ensure that.
1. Every task is identified and listed.
2. Add prerequisite tasks also in case required. 
3. Add tasks to handle the alternate and exception scenarios as well.
4. Explain every task in descriptive manner in step-by-step.
5. Consider all technical details and their related actions also include integer or numeric values from jira-feature-details.
6. Look for term "ADF", if exists then create tasks otherwise skip this step.
7. Look for term "Control-M", if exists then create tasks otherwise skip this step.
                          
consider below mentioned points as well while creating tasks
1. Change analysis 
2. LLD 
3. Accessibility
4. Deployment
5. Release notes preparation Ops handoff 
6. Build and pipeline creation
7. Security scans and vulnerability fixes
8. Performance testing
9. Test documentation and Automation
10. UAT test Support
11. NOD Support and BVT
12. Environment Sync up (True up) activity
                           
<jira-feature-details>
 """ + org_requirements + """                   
</jira-feature-details>
                                              
""")

tasks_user_prompt = str("""
 Please help identify that tasks.
""")

messages = [
    {"role": "system", "content": tasks_system_prompt},    
    {"role": "user", "content": tasks_user_prompt}
]

code = [""]

result=openai.ChatCompletion.create(
# result=openai.Completion.create(
   model= model, 
   messages=messages,
   engine=deployment_name,
    temperature=temperature,
   top_p=top_p
   
)

print("---- Tokens------")
print(f"completion tokens- {result.usage.completion_tokens}")
print(f"prompt tokens- {result.usage.prompt_tokens}")
print(f"Total tokens- {result.usage.total_tokens}")

res = result.choices[0].message.content
task_assistant = str(res)
print(f"Response: {res}")
code.append(f"{res}")

---- Tokens------
completion tokens- 583
prompt tokens- 654
Total tokens- 1237
Response: 1. **Change Analysis:**
   Analyze the existing system and identify the components that need to be changed or decommissioned. Understand the data extraction, formatting, encryption, and transfer processes of RECON job on the old system. 

2. **Low Level Design (LLD):** 
   Design the Azure Data Factory (ADF) and Azure Databricks (ADB) pipeline architecture to extract data from the new data source, format it into a vendor-specific layout, encrypt the RECON file using the New Analytics Public Key, and store the file in Azure-specific storage.

3. **Accessibility:**
   Ensure the accessibility of the new data source, Azure-specific storage, and vendor's SFTP server. Provision necessary access rights and permissions.

4. **Build and pipeline creation:**
   Implement the designed ADF and ADB pipeline. This includes writing scripts for data extraction, formatting, encryption, and transfer to Azure-specif

In [50]:
# User Story from the task identified.
temperature=1
top_p=0.35
    
partial_task = [task_assistant[i:i+5] for i in range(0, len(task_assistant),5)]
for task in partial_task:
    user_story_system_prompt = str("""
Please assume the role of a product owner for T-Mobile, a prominent telecommunications provider based in North America. You have been given the following inputs that detail the necessary technical changes:

Technical details provided as input in the Jira feature as .
High-level tasks that need to be executed for the technical change as .
Your task is to create user stories and include the following details for each identified user story:
1. Summary
2. Description
3. Technical Details - Be sure to extract all the technical details from the inputs, including any numerical values, and add them to this section in a descriptive manner.
4. Acceptance Criteria
5. Sequence of execution and dependency
Also, please add additional stories for:
1. Creation of a Low-Level Design Document
2. Customer approval of the Low-Level Design Document
3. DevOps and Release
4. Support and User Acceptance Testing (UAT)
5. Preparation of release notes and operational activity
When providing the list of user stories, please ensure:
1. You must provide all the user stories, without writing extra explanations.
2. All user stories contain the necessary details.
3. The list of user stories is sorted based on priority and dependency.
add '------------------------------------------------------------------------------------' after every Sequence of execution and dependency to separate.

                              
<jira-feature-details>
 """ + org_requirements + """                   
</jira-feature-details> 

<tasks-list>
 """ + "".join(task) + """                   
</tasks-list>                                               
""")

user_story_user_prompt = str("""
 Please create user stories.
""")

messages = [
    {"role": "system", "content": user_story_system_prompt},    
    {"role": "user", "content": user_story_user_prompt}
]

code = [""]

result=openai.ChatCompletion.create(
# result=openai.Completion.create(
   model= model, 
   messages=messages,
   max_tokens = 8192,
   engine=deployment_name,
   temperature=temperature,
   top_p=top_p
)


print("---- Tokens------")
print(f"completion tokens- {result.usage.completion_tokens}")
print(f"prompt tokens- {result.usage.prompt_tokens}")
print(f"Total tokens- {result.usage.total_tokens}")

res = result.choices[0].message.content
#log_message(user_id, f"Generated Story: {res}")
user_story_assistant = str(res)
print(f"Response: {res}")
code.append(f"{res}") 

---- Tokens------
completion tokens- 626
prompt tokens- 656
Total tokens- 1282
Response: User Story 1:
Summary: Decommission Old Data Platform - RECON job
Description: As a product owner, I want to decommission the old system and move the entire process to Azure cloud to leverage data bricks for extracting data from the new data source.
Technical Details: The current system uses OLD SYSTEM to extract data from AUX_BILLING_ACCOUNT and transfer it to NEW ANALYTICS. The plan is to move this process to Azure cloud and use data bricks to extract data from the new data source.
Acceptance Criteria: The old system is successfully decommissioned and the new process using Azure cloud and data bricks is implemented and functioning as expected.
Sequence of execution and dependency: This is the first task and has no dependencies.
------------------------------------------------------------------------------------

User Story 2:
Summary: Creation of a Low-Level Design Document
Description: As a prod

In [51]:
import requests
from requests.auth import HTTPBasicAuth
import json

# Replace these with your Jira credentials and project details
JIRA_URL = 'https://advantagealm.atlassian.net/rest/agile/latest/epic/TGP-676/issue'
JIRA_AUTH = ('sruthiv@hcltech.com', 'ATATT3xFfGF0RsQ0V1dYM9P9IRFUBaVv3cp1SvBFayGcm9bTVWjIdrNQSMfJYTQTkgIKxbsEr_6knzvtVgVenMF9EMhClZUOBnCBYEyvuDXpuxHCA8ATECsUbPjGETz4ZzHW95AgRJFzd-08yNg63tP8-oAw-1if6nRGJQa90h2KsdeY8XvWXeg=63B65EE3')

# Replace these variables with your details
#JIRA_URL = 'https://your-domain.atlassian.net'
USERNAME = 'sruthiv@hcltech.com'
API_TOKEN = 'ATATT3xFfGF0RsQ0V1dYM9P9IRFUBaVv3cp1SvBFayGcm9bTVWjIdrNQSMfJYTQTkgIKxbsEr_6knzvtVgVenMF9EMhClZUOBnCBYEyvuDXpuxHCA8ATECsUbPjGETz4ZzHW95AgRJFzd-08yNg63tP8-oAw-1if6nRGJQa90h2KsdeY8XvWXeg=63B65EE3'
EPIC_KEY = 'TGP-676'



# Headers for the request
headers = {
    'Accept': 'application/json'
}

# Make the GET request
response = requests.get(
    JIRA_URL,
    headers=headers,
    auth=HTTPBasicAuth(USERNAME, API_TOKEN)
)
# List to store issues
linked_issues = []

# Check if the request was successful
if response.status_code == 200:
    issues = response.json()['issues']
    for issue in issues:
        linked_issues.append({
            'issue_key': issue['key'],
            'summary': issue['fields']['summary']
        })
else:
    print(f"Failed to fetch issues: {response.status_code} - {response.text}")

linked_issues_str = json.dumps(linked_issues)
print("\n linked_issues_str : ",linked_issues_str) 


# Print stored issues
for issue in linked_issues:
    print(f"Issue Key: {issue['issue_key']}, Summary: {issue['summary']}")





 linked_issues_str :  [{"issue_key": "TGP-677", "summary": "Create JIRA Story with Rest API:Setup Azure Databricks Environmen"}, {"issue_key": "TGP-678", "summary": "Data Extraction from Snowflake ."}, {"issue_key": "TGP-679", "summary": "Data Formatting ."}, {"issue_key": "TGP-680", "summary": "File Encryption ."}, {"issue_key": "TGP-681", "summary": "Support and User Acceptance Testing (UAT)"}, {"issue_key": "TGP-682", "summary": "Preparation of release notes and operational activity"}, {"issue_key": "TGP-683", "summary": "DevOps and Release"}, {"issue_key": "TGP-735", "summary": "Decommission Old Data Platform - RECON job\",         \"   ."}, {"issue_key": "TGP-736", "summary": "Creation of a Low-Level Design Document\",         \"   ."}, {"issue_key": "TGP-737", "summary": "Customer approval of the Low-Level Design Document\",         \"   ."}, {"issue_key": "TGP-738", "summary": "Support and User Acceptance Testing (UAT)\",         \"   ."}, {"issue_key": "TGP-739", "summary": "P

In [65]:
# User Story from the task identified.
temperature=1
top_p=0.35
#linked_issues_str :  [{"issue_key": "TGP-677", "summary": "Decommission Old Data Platform - RECON job"}, {"issue_key": "TGP-678", "summary": "Data Extraction from Snowflake ."}, {"issue_key": "TGP-679", "summary": "Support and User Acceptance Testing (UAT)"}, {"issue_key": "TGP-680", "summary": "File Encryption ."}, {"issue_key": "TGP-681", "summary": "Data Extraction from Snowflake ."}, {"issue_key": "TGP-682", "summary": "Data Formatting ."}, {"issue_key": "TGP-683", "summary": "File Encryption ."}]

duplicate_story_system_prompt = str("""
Please compare both provided details jira-feature-details and task-list on the base of summary and list out the duplicated stories and give 2 lists :
1. dupliated stories
2. new stories 
Your task is to add  User Story, Summary, Description, Technical Details - Be sure to extract all the technical details from the inputs, including any numerical values, and add them to this section in a descriptive manner.
 ,Acceptance Criteria, Sequence of execution and dependency from <jira-feature-details> for each 'new stories'.
                                                               
<jira-feature-details>
 """ + user_story_assistant + """                   
</jira-feature-details> 

<tasks-list>
 """ + linked_issues_str + """                   
</tasks-list>                                               
""")

duplicate_story_user_prompt = str("""
 Please list out the duplicated and non-duplicated user stories based on 'summary'.
""")

messages = [
    {"role": "system", "content": duplicate_story_system_prompt},    
    {"role": "user", "content": duplicate_story_user_prompt}
]


result=openai.ChatCompletion.create(
# result=openai.Completion.create(
   model= model, 
   messages=messages,
   max_tokens = 8192,
   engine=deployment_name,
   temperature=temperature,
   top_p=top_p
)

print("---- Tokens------")
print(f"completion tokens- {result.usage.completion_tokens}")
print(f"prompt tokens- {result.usage.prompt_tokens}")
print(f"Total tokens- {result.usage.total_tokens}")
#print("result ::\n\n", result)

res = result.choices[0].message.content
#log_message(user_id, f"Generated Story: {res}")
user_story_assistant = str(res)
print(f"Response: {res}")
code.append(f"{res}") 

# Splitting the input string into sections
sections = res.strip().split("\n\n")

# Check the length of sections before accessing them
if len(sections) > 1:
    duplicated_stories_section = sections[0]
    new_stories_section = sections[1].split('\n')
    
    # Further processing of new_stories_section
    for line in new_stories_section:
        print(line)
else:
    print("Error: The input string does not contain the expected sections.")



---- Tokens------
completion tokens- 175
prompt tokens- 637
Total tokens- 812
Response: Duplicated Stories:

1. Summary: Support and User Acceptance Testing (UAT)
2. Summary: Preparation of release notes and operational activity

New Stories:

1. Summary: Create JIRA Story with Rest API:Setup Azure Databricks Environment
2. Summary: Data Extraction from Snowflake.
3. Summary: Data Formatting.
4. Summary: File Encryption.
5. Summary: DevOps and Release
6. Summary: Decommission Old Data Platform - RECON job
7. Summary: Creation of a Low-Level Design Document
8. Summary: Customer approval of the Low-Level Design Document

Note: The new stories do not have the additional details such as User Story, Description, Technical Details, Acceptance Criteria, and Sequence of execution and dependency. These details need to be added from the <jira-feature-details> for the new stories.
1. Summary: Support and User Acceptance Testing (UAT)
2. Summary: Preparation of release notes and operational activi

In [53]:
# Split the input text into sections
sections = res.split('\n\nNew Stories:\n')
duplicate_stories_section = sections[0].split('\n')[1:]
new_stories_section = sections[1].split('\n')

# Store duplicate stories
duplicate_stories = []
for line in duplicate_stories_section:
    if line.startswith('1.') or line.startswith('2.') or line.startswith('3.'):
        duplicate_stories.append(line.strip())

# Store new stories
new_stories = []
current_story = {}
for line in new_stories_section:
    if line.startswith('1. Summary:') or line.startswith('2. Summary:') or line.startswith('3. Summary:'):
        if current_story:
            new_stories.append(current_story)
        current_story = {'Summary': line.split('Summary: ')[1].strip()}
    elif line.startswith('   Description:'):
        current_story['Description'] = line.split('Description: ')[1].strip()
    elif line.startswith('   Technical Details:'):
        current_story['Technical Details'] = line.split('Technical Details: ')[1].strip()
    elif line.startswith('   Acceptance Criteria:'):
        current_story['Acceptance Criteria'] = line.split('Acceptance Criteria: ')[1].strip()
    elif line.startswith('   Sequence of execution and dependency:'):
        current_story['Sequence of execution and dependency'] = line.split('Sequence of execution and dependency: ')[1].strip()

if current_story:
    new_stories.append(current_story)

# Create JSON structure
output_data = {
    'Duplicated Stories': duplicate_stories,
    'New Stories': new_stories
}

# Convert to JSON
json_output = json.dumps(output_data, indent=4)

# Output the JSON to UI (for example, you can print it or send it through a web framework)
#print(json_output)
newstories =  output_data.get('New Stories', [])

print("New stories : ",newstories)

# formatted_stories = {}

# for index, story in enumerate(newstories, start=1):
#     key = f"User Story {index}"
#     formatted_stories[key] = story
#     print("\nformatted_stories[key] :", formatted_stories)
# for key, value in formatted_stories.items():
#     print(f"{key}: {value}\n")
#     print("\ndata type of F-stories :", type(formatted_stories))

#     dict_as_simple_string = str(formatted_stories)

# # Print the string
# print(dict_as_simple_string)
# print("\ndata type of dict_as_simple_string :", type(dict_as_simple_string))

New stories :  [{'Summary': 'Create JIRA Story with Rest API:Setup Azure Databricks Environment'}, {'Summary': 'Data Extraction from Snowflake.'}, {'Summary': 'Data Formatting.'}]


In [54]:
import re

# Given data
data = """
1. Summary: Decommission of Old Data Platform - RECON job
   Description: As a Product Owner, I want to decommission the OLD SYSTEM used for RECON job, and migrate the process to AZURE cloud leveraging data bricks to extract data from the new data source. This is to enhance the process efficiency and system stability.
   Technical Details: The OLD SYSTEM is currently used to extract data from AUX_BILLING_ACCOUNT, format it, and transfer to NEW ANALYTICS. The new process will use Azure Data Factory (ADF) and Azure Databricks (ADB) to extract data from the new data source, format it as per Vendor's specification, and encrypt the file using New Analytics Public Key. The file will then be transferred to the vendor via Control-M, which should not delete the file during Soft Launch Mode.
   Acceptance Criteria: The extraction of RECON data from the new data source, formatting it into the Vendor suggested layout, encryption of the RECON file, and successful transfer to NEW ANALYTICS is successful.
   Sequence of Execution and Dependency: This user story should be executed first as it forms the basis for subsequent user stories.

2. Summary: Creation of a Low-Level Design Document
   Description: As a Product Owner, I need a Low-Level Design (LLD) document that outlines the solution architecture for the new process, including data extraction, formatting, encryption, and transfer.
   Technical Details: The LLD should include details about the Azure infrastructure, ADF+ADB pipeline, Control-M job, storage location for formatted data, encryption process, and alerts & notifications mechanism.
   Acceptance Criteria: A comprehensive LLD is created which covers all the technical aspects for the decommissioning of the OLD SYSTEM and setting up the new process.
   Sequence of Execution and Dependency: This user story is dependent on User Story 1 and should be executed after it.

3. Summary: Customer approval of the Low-Level Design Document
   Description: As a Product Owner, I need approval from the customer on the LLD which outlines the new process.
   Technical Details: The approval process involves presenting the LLD to the customer and making necessary revisions based on their feedback.
   Acceptance Criteria: The LLD is approved by the customer.
   Sequence of Execution and Dependency: This user story is dependent on User Story 2 and should be executed after it.

4. Summary: Support and User Acceptance Testing (UAT)
   Description: As a Product Owner, I need the solution to be tested by the UAT team and any issues or defects reported during UAT addressed.
   Technical Details: The UAT process involves creating test cases, automating them using appropriate tools, executing the tests, and addressing any issues or defects that arise.
   Acceptance Criteria: The solution passes UAT without any critical issues or defects.
   Sequence of Execution and Dependency: This user story is dependent on User Story 3 and should be executed after it.

5. Summary: Preparation of release notes and operational activity
   Description: As a Product Owner, I need comprehensive release notes documenting the system change, its impact, steps involved, known issues, and limitations. Also, I need a successful handoff session with the operations team.
   Technical Details: The release notes should include all the details about the change in the system, the impact, and the steps involved. The handoff session should ensure that the operations team is fully briefed and prepared to manage the new system.
   Acceptance Criteria: Release notes are comprehensive, and a successful handoff session is conducted with the operations team.
   Sequence of Execution and Dependency: This user story is dependent on User Story 4 and should be executed after it.
"""

def add_user_story_index(data):
    # Split the data by newlines
    lines = data.strip().split('\n')
    
    # Initialize an index counter
    index_counter = 1
    
    # Prepare a list to hold the modified user stories
    indexed_stories = []
    
    # Iterate through the lines
    for line in lines:
        # Check if the line starts with a number followed by a period
        if re.match(r'^\d+\. Summary:', line):
            # If it's a summary line, add the index text before it
            indexed_stories.append(f'User story {index_counter} : ')
            indexed_stories.append(line)
            index_counter += 1
        else:
            # If it's not a summary line, just append it as is
            indexed_stories.append(line)
    
    # Join the indexed stories back into a single string
    return '\n'.join(indexed_stories)

# Add User Story indexes to the given data
indexed_data = add_user_story_index(data)

# Print the modified data
print(indexed_data)


User story 1 : 
1. Summary: Decommission of Old Data Platform - RECON job
   Description: As a Product Owner, I want to decommission the OLD SYSTEM used for RECON job, and migrate the process to AZURE cloud leveraging data bricks to extract data from the new data source. This is to enhance the process efficiency and system stability.
   Technical Details: The OLD SYSTEM is currently used to extract data from AUX_BILLING_ACCOUNT, format it, and transfer to NEW ANALYTICS. The new process will use Azure Data Factory (ADF) and Azure Databricks (ADB) to extract data from the new data source, format it as per Vendor's specification, and encrypt the file using New Analytics Public Key. The file will then be transferred to the vendor via Control-M, which should not delete the file during Soft Launch Mode.
   Acceptance Criteria: The extraction of RECON data from the new data source, formatting it into the Vendor suggested layout, encryption of the RECON file, and successful transfer to NEW ANA

In [55]:
import re

# Given data
raw_data = [
    "1. Summary: Decommission of Old Data Platform - RECON job",
    "   Description: As a Product Owner, I want to decommission the OLD SYSTEM used for RECON job, and migrate the process to AZURE cloud leveraging data bricks to extract data from the new data source. This is to enhance the process efficiency and system stability.",
    "   Technical Details: The OLD SYSTEM is currently used to extract data from AUX_BILLING_ACCOUNT, format it, and transfer to NEW ANALYTICS. The new process will use Azure Data Factory (ADF) and Azure Databricks (ADB) to extract data from the new data source, format it as per Vendor's specification, and encrypt the file using New Analytics Public Key. The file will then be transferred to the vendor via Control-M, which should not delete the file during Soft Launch Mode.",
    "   Acceptance Criteria: The extraction of RECON data from the new data source, formatting it into the Vendor suggested layout, encryption of the RECON file, and successful transfer to NEW ANALYTICS is successful. ",
    "   Sequence of Execution and Dependency: This user story should be executed first as it forms the basis for subsequent user stories.",
    "",
    "2. Summary: Creation of a Low-Level Design Document",
    "   Description: As a Product Owner, I need a Low-Level Design (LLD) document that outlines the solution architecture for the new process, including data extraction, formatting, encryption, and transfer.",
    "   Technical Details: The LLD should include details about the Azure infrastructure, ADF+ADB pipeline, Control-M job, storage location for formatted data, encryption process, and alerts & notifications mechanism.",
    "   Acceptance Criteria: A comprehensive LLD is created which covers all the technical aspects for the decommissioning of the OLD SYSTEM and setting up the new process.",
    "   Sequence of Execution and Dependency: This user story is dependent on User Story 1 and should be executed after it.",
    "",
    "3. Summary: Customer approval of the Low-Level Design Document",
    "   Description: As a Product Owner, I need approval from the customer on the LLD which outlines the new process.",
    "   Technical Details: The approval process involves presenting the LLD to the customer and making necessary revisions based on their feedback.",
    "   Acceptance Criteria: The LLD is approved by the customer.",
    "   Sequence of Execution and Dependency: This user story is dependent on User Story 2 and should be executed after it.",
    "",
    "4. Summary: Support and User Acceptance Testing (UAT)",
    "   Description: As a Product Owner, I need the solution to be tested by the UAT team and any issues or defects reported during UAT addressed.",
    "   Technical Details: The UAT process involves creating test cases, automating them using appropriate tools, executing the tests, and addressing any issues or defects that arise.",
    "   Acceptance Criteria: The solution passes UAT without any critical issues or defects.",
    "   Sequence of Execution and Dependency: This user story is dependent on User Story 4 and should be executed after it.",
    "",
    "5. Summary: Preparation of release notes and operational activity",
    "   Description: As a Product Owner, I need comprehensive release notes documenting the system change, its impact, steps involved, known issues, and limitations. Also, I need a successful handoff session with the operations team.",
    "   Technical Details: The release notes should include all the details about the change in the system, the impact, and the steps involved. The handoff ses        ",
    "   Technical Details: The UAT process involves creating test cases, automating them using appropriate tools, executing the tests, and addressing any issues or defects that arise.",
    "   Acceptance Criteria: The solution passes UAT without any critical issues or defects.",
    "   Sequence of Execution and Dependency: This user story is dependent on User Story 4 and should be executed after it.",
    "",
    "5. Summary: Preparation of release notes and operational activity",
    "   Description: As a Product Owner, I need comprehensive release notes documenting the system change, its impact, steps involved, known issues, and limitations. Also, I need a successful handoff session with the operations team.",
    "   Technical Details: The release notes should include all the details about the change in the system, the impact, and the steps involved. The handoff ses        ",
    "   Acceptance Criteria: The solution passes UAT without any critical issues or defects.",
    "   Sequence of Execution and Dependency: This user story is dependent on User Story 4 and should be executed after it.",
    "",
    "5. Summary: Preparation of release notes and operational activity",
    "   Description: As a Product Owner, I need comprehensive release notes documenting the system change, its impact, steps involved, known issues, and limitations. Also, I need a successful handoff session with the operations team.",
    "   Technical Details: The release notes should include all the details about the change in the system, the impact, and the steps involved. The handoff ses        ",
    "5. Summary: Preparation of release notes and operational activity",
    "   Description: As a Product Owner, I need comprehensive release notes documenting the system change, its impact, steps involved, known issues, and limitations. Also, I need a successful handoff session with the operations team.",
    "   Technical Details: The release notes should include all the details about the change in the system, the impact, and the steps involved. The handoff ses        ",
    "   Description: As a Product Owner, I need comprehensive release notes documenting the system change, its impact, steps involved, known issues, and limitations. Also, I need a successful handoff session with the operations team.",
    "   Technical Details: The release notes should include all the details about the change in the system, the impact, and the steps involved. The handoff sestions. Also, I need a successful handoff session with the operations team.",
    "   Technical Details: The release notes should include all the details about the change in the system, the impact, and the steps involved. The handoff ses        ",
    "   Technical Details: The release notes should include all the details about the change in the system, the impact, and the steps involved. The handoff session with the operations team should include a detailed explanation of the changes and necessary documentation.",
    "   Acceptance Criteria: Comprehensive release notes are prepared, and a successful handoff session is conducted with the operations team.",
    "   Sequence of Execution and Dependency: This user story is dependent on User Story 5 and should be executed after it."
]

# Function to add user story index before every summary
def add_index_to_summaries(data):
    formatted_data = []
    index = 1
    
    for line in data:
        line = line.strip()
        # Check if the line starts with a number followed by a period
        if re.match(r'^\d+\.', line):
            # Add user story index to the summary
            formatted_data.append(f"User Story {index}: {line}")
            index += 1
        else:
            # Add other lines without modification
            formatted_data.append(line)
    
    return formatted_data

# Add index to summaries
formatted_data = add_index_to_summaries(raw_data)

# Print the formatted data
for line in formatted_data:
    print(line)


User Story 1: 1. Summary: Decommission of Old Data Platform - RECON job
Description: As a Product Owner, I want to decommission the OLD SYSTEM used for RECON job, and migrate the process to AZURE cloud leveraging data bricks to extract data from the new data source. This is to enhance the process efficiency and system stability.
Technical Details: The OLD SYSTEM is currently used to extract data from AUX_BILLING_ACCOUNT, format it, and transfer to NEW ANALYTICS. The new process will use Azure Data Factory (ADF) and Azure Databricks (ADB) to extract data from the new data source, format it as per Vendor's specification, and encrypt the file using New Analytics Public Key. The file will then be transferred to the vendor via Control-M, which should not delete the file during Soft Launch Mode.
Acceptance Criteria: The extraction of RECON data from the new data source, formatting it into the Vendor suggested layout, encryption of the RECON file, and successful transfer to NEW ANALYTICS is s

In [56]:

# #JIRA INsertion of new stories 
from jira import JIRA

# JIRA credentials and server
# Replace these with your Jira credentials and project details
JIRA_URL = 'https://advantagealm.atlassian.net/rest/api/latest/issue'
JIRA_AUTH = ('sruthiv@hcltech.com', '-08yNg63tP8-oAw-1if6nRGJQa90h2KsdeY8XvWXeg=63B65EE3')
jira_user = 'sruthiv@hcltech.com'
jira_api_token = 'ATATT3xFfGF0RsQ0V1dYM9P9IRFUBaVv3cp1SvBFayGcm9bTVWjIdrNQSMfJYTQTkgIKxbsEr_6knzvtVgVenMF9EMhClZUOBnCBYEyvuDXpuxHCA8ATECsUbPjGETz4ZzHW95AgRJFzd'

PROJECT_KEY = '10284'  # Replace with your Jira project key
def create_jira_issue(summary, description, technical_details, acceptance_criteria):
    url = JIRA_URL
    headers = {
        "Content-Type": "application/json"
    }
    auth = JIRA_AUTH

    payload = {
        'fields': {
            'project': {
                "id": "10284",
                "key": "TGP",
                "name": "TMO-GenAI POC"
            },
            'summary': summary,
            'description': f"{description}\n\nTechnical Details:\n{technical_details}\n\nAcceptance Criteria:\n{acceptance_criteria}",
            'issuetype': {
                'name': 'Task'
            }
        }
    }

   
    response = requests.post(url, headers=headers, auth=auth ,json=payload, verify=False)

    if response.status_code == 201:
        print(f"Issue created successfully: {response.json()['key']}")
    else:
        print(f"Failed to create issue: {response.text}")

# Insert new stories into JIRA
for story in new_stories:
    create_jira_issue(
        summary=story.get('summary'),
        description=story.get('description'),
        technical_details=story.get('technical_details', 'N/A'),
        acceptance_criteria=story.get('acceptance_criteria')
    )

c:\Users\agrita.verma\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'advantagealm.atlassian.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Failed to create issue: {"errorMessages":["You do not have permission to create issues in this project."],"errors":{}}


c:\Users\agrita.verma\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'advantagealm.atlassian.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Failed to create issue: {"errorMessages":["You do not have permission to create issues in this project."],"errors":{}}


c:\Users\agrita.verma\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'advantagealm.atlassian.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Failed to create issue: {"errorMessages":["You do not have permission to create issues in this project."],"errors":{}}


In [57]:
import re

# user_stories_str = """
# User Story 1:
# 1. Summary: Decommission Old Data Platform - RECON job
# 2. Description: Extract RECON data from new data source, format data into Vendor suggested layout, encrypt recon file, and transfer to NEW ANALYTICS.
# 3. Technical Details: The new system will leverage Azure cloud and data bricks to extract data from the new data source. The data will be formatted into a vendor-suggested layout and encrypted. The file will be named "Recon_DDMMYYYY.txt" and transferred to NEW ANALYTICS.
# 4. Acceptance Criteria: The RECON data is successfully extracted from the new data source, formatted, encrypted, and transferred to NEW ANALYTICS.
# 5. Sequence of execution and dependency: This is the first task and has no dependencies.

# User Story 2:
# 1. Summary: Creation of a Low-Level Design Document
# 2. Description: Create a Low-Level Design Document detailing the new system and its functionality.
# 3. Technical Details: The document will detail the Azure-based solution (ADF+ADB), the extraction of data from the new data source, the formatting of data, encryption, and transfer to NEW ANALYTICS.
# 4. Acceptance Criteria: A comprehensive Low-Level Design Document is created.
# 5. Sequence of execution and dependency: This task depends on User Story 1.

# User Story 3:
# 1. Summary: Customer approval of the Low-Level Design Document
# 2. Description: Obtain customer approval for the Low-Level Design Document.
# 3. Technical Details: Present the Low-Level Design Document to the customer for review and approval.
# 4. Acceptance Criteria: The customer approves the Low-Level Design Document.
# 5. Sequence of execution and dependency: This task depends on User Story 2.
# """

def parse_user_stories(user_story_assistant):
    user_stories = []
    stories = user_story_assistant.strip().split("\n\n")
    
    for story in stories:
        lines = story.split("\n")
        summary = re.search(r"Summary: (.*)", lines[1]).group(1)
        user_stories.append({
            "ID": lines[0].replace("User Story ", "").strip(":"),
            "Summary": summary,
            "Description": lines[2].split(": ", 1)[1],
            "Technical Details": lines[3].split(": ", 1)[1],
            "Acceptance Criteria": lines[4].split(": ", 1)[1],
            "Sequence of execution and dependency": lines[5].split(": ", 1)[1]
        })
    return user_stories

user_stories = parse_user_stories(user_story_assistant)
print(user_stories)


IndexError: list index out of range

In [ ]:
def find_duplicates(user_stories, linked_issues):
    #print("issue",linked_issues["Summary"])
    linked_summaries = {issue['summary'].strip(): issue['issue_key'] for issue in linked_issues}
    print("linked_summaries",linked_summaries)
    duplicates = []

    for story in user_stories:
        if story["Summary"].strip() in linked_summaries:
            duplicates.append(story["ID"])
            print(f"Duplicate found: User Story ID {story['ID']} has the same summary as Linked Issue {linked_summaries[story['Summary'].strip()]}")

    return duplicates

duplicates = find_duplicates(user_stories, linked_issues)
print(f"Duplicates: {duplicates}")


In [ ]:
def delete_user_stories(user_stories, duplicates):
    remaining_stories = [story for story in user_stories if story["ID"] not in duplicates]
    return remaining_stories

remaining_user_stories = delete_user_stories(user_stories, duplicates)
print(f"Remaining User Stories: {remaining_user_stories}")
